In [1]:
import pandas as pd
import os
import re
from shutil import copyfile

In [2]:
#Read files
xls = pd.ExcelFile('BIDS_info.xlsx')

#path variables
BIDS='./BIDS_data/'
NIMS='./NIMS_data/'

#Make folder if folder doesn't exist function
def makefolder(name):
    if not os.path.exists(name):
        os.makedirs(name)

In [3]:
participants = xls.parse('participants')
protocol = xls.parse('protocol', convert_float=False).iloc[1:,:4] #columns 5 on are reference columns
protocol = protocol.dropna(axis=0, thresh=3) #get rid of items that don't have a bids equivalent
protocol.run_number = protocol.run_number.astype('str').str.strip('.0')
protocol["BIDS_scan_title_path"] = BIDS + "sub-###/" + protocol.ANAT_or_FUNCT + "/sub-###_" + protocol.BIDS_scan_title + "_run-" + protocol.run_number + ".nii.gz"

protocol.BIDS_scan_title_path = protocol.BIDS_scan_title_path.str.replace('_run-nan', '')

NIMS_dir_ref = protocol.NIMS_scan_title.tolist()

NIMS_BIDS_conversion = protocol[[0,4]]

In [4]:
#Create Participants folders in BIDS
makefolder(BIDS)
participants.participant_id.apply(lambda x: makefolder(BIDS + 'sub-' + str(x) + "/anat"))
participants.participant_id.apply(lambda x: makefolder(BIDS + 'sub-' + str(x) + "/funct"))

0    None
1    None
2    None
3    None
4    None
Name: participant_id, dtype: object

In [5]:
def check_against_protocol(participants,protocol): 
    
    all_files_correct = True

    for index, row in participants.iterrows():
        #Get files
        NIMS_dir_files = os.listdir(NIMS + row.nims_title)

        #Delete all non-nii.gz files
        NIMS_dir_files = [x for x in NIMS_dir_files if ".nii.gz"  in x]

        for item in set(NIMS_dir_ref):
            directory_filenames = [x for x in NIMS_dir_files if item in x]
            protocol_filenames = NIMS_BIDS_conversion[NIMS_BIDS_conversion.NIMS_scan_title.str.contains(item)]
            protocol_filenames = protocol_filenames.iloc[:,1].tolist()
            
            if len(directory_filenames) == len(protocol_filenames):
                print("sub-" + str(row.participant_id) + ": ++ " + item.rjust(20) + " match")

            else:
                print("sub-" + str(row.participant_id) + ": -- "+ item.rjust(20) + " files do not match protocol")
                all_files_correct = False
        print("------------")
        
        
    print("All your folders match your protocol") if all_files_correct else print("Some folders do not match your protocol, please resolve errors")

In [6]:
check_against_protocol(participants, protocol)

sub-100: ++   BOLD_EPI_29mm_2sec match
sub-100: ++        T1w_9mm_BRAVO match
------------
sub-101: ++   BOLD_EPI_29mm_2sec match
sub-101: ++        T1w_9mm_BRAVO match
------------
sub-102: --   BOLD_EPI_29mm_2sec files do not match protocol
sub-102: ++        T1w_9mm_BRAVO match
------------
sub-103: ++   BOLD_EPI_29mm_2sec match
sub-103: ++        T1w_9mm_BRAVO match
------------
sub-104: ++   BOLD_EPI_29mm_2sec match
sub-104: ++        T1w_9mm_BRAVO match
------------
Some folders do not match your protocol, please resolve errors


In [15]:
def rename_to_bids(participants, protocol):

    for index, row in participants.iterrows():
        #Get files
        NIMS_dir_files = os.listdir(NIMS + row.nims_title)

        #Delete all non-nii.gz files
        NIMS_dir_files = [x for x in NIMS_dir_files if ".nii.gz"  in x]

        for item in set(NIMS_dir_ref):
            directory_filenames = [x for x in NIMS_dir_files if item in x]
            protocol_filenames = NIMS_BIDS_conversion[NIMS_BIDS_conversion.NIMS_scan_title.str.contains(item)]
            protocol_filenames = protocol_filenames.iloc[:,1].tolist()
            
            assert len(directory_filenames) == len(protocol_filenames), "filelists aren't the same length"
            
            for index, item in enumerate(directory_filenames):
                oldpath = (NIMS + row.nims_title + "/" + directory_filenames[index])
                newpath = (protocol_filenames[index].replace("###", str(row.participant_id)))
                copyfile(oldpath, newpath)
        
                print("sub-" + str(row.participant_id) + item.rjust(20))
        print("------------")
        


In [16]:
rename_to_bids(participants, protocol)

sub-100: ++ 0004_01_BOLD_EPI_29mm_2sec.nii.gz
sub-100: ++ 0005_01_BOLD_EPI_29mm_2sec.nii.gz
sub-100: ++ 0006_01_BOLD_EPI_29mm_2sec.nii.gz
sub-100: ++ 0007_01_BOLD_EPI_29mm_2sec.nii.gz
sub-100: ++ 0008_01_T1w_9mm_BRAVO.nii.gz
------------
sub-101: ++ 0005_01_BOLD_EPI_29mm_2sec.nii.gz
sub-101: ++ 0006_01_BOLD_EPI_29mm_2sec.nii.gz
sub-101: ++ 0007_01_BOLD_EPI_29mm_2sec.nii.gz
sub-101: ++ 0008_01_BOLD_EPI_29mm_2sec.nii.gz
sub-101: ++ 0009_01_T1w_9mm_BRAVO.nii.gz
------------
sub-102: ++ 0004_01_BOLD_EPI_29mm_2sec.nii.gz
sub-102: ++ 0005_01_BOLD_EPI_29mm_2sec.nii.gz
sub-102: ++ 0006_01_BOLD_EPI_29mm_2sec.nii.gz
sub-102: ++ 0007_01_BOLD_EPI_29mm_2sec.nii.gz
sub-102: ++ 0008_01_T1w_9mm_BRAVO.nii.gz
------------
sub-103: ++ 0004_01_BOLD_EPI_29mm_2sec.nii.gz
sub-103: ++ 0005_01_BOLD_EPI_29mm_2sec.nii.gz
sub-103: ++ 0006_01_BOLD_EPI_29mm_2sec.nii.gz
sub-103: ++ 0007_01_BOLD_EPI_29mm_2sec.nii.gz
sub-103: ++ 0008_01_T1w_9mm_BRAVO.nii.gz
------------
sub-104: ++ 0004_01_BOLD_EPI_29mm_2sec.nii.gz
su